In [ ]:
from pathlib import Path

# import typing as t
from IPython.display import Markdown

from kedro.config import OmegaConfigLoader
from kedro.framework.project import settings
from google import genai

# from google.genai import types

In [ ]:
# configure credentials
conf_path = str(Path("..") / settings.CONF_SOURCE)
conf_loader = OmegaConfigLoader(conf_source=conf_path)
GOOGLE_API_KEY = conf_loader["credentials"]["google_api_credentials"]["key"]

### Purpose of this Notebook

The purpose of this notebook is to convert a resume from PDF format to JSON format, adhering to the JSON schema proposed by this page: [JSON Resume Schema](https://jsonresume.org/schema).

In [ ]:
pdf_cv: str = catalog.load("pdf_resume")  # noqa: F821
Markdown(pdf_cv)

In [ ]:
client = genai.Client(api_key=GOOGLE_API_KEY)